In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('SalesDataPrep').getOrCreate()

In [0]:
path = '/FileStore/tables/salesdata'

In [0]:
from pyspark.sql.types import StructType,StructField
from pyspark.sql.types import StringType,FloatType,IntegerType

In [0]:
schema = StructType([
    StructField('orderId',StringType(),True),
    StructField('product',StringType(),True),
    StructField('quantityOrdered',StringType(),True),
    StructField('priceEach',StringType(),True),
    StructField('orderDate',StringType(),True),
    StructField('storeAddress',StringType(),True)
])

In [0]:
sales_dataframe = spark.read.csv(path = path, header=True, schema = schema)
sales_dataframe.show(10,False)

+-------+--------------------------+---------------+---------+--------------+-----------------------------------------+
|orderId|product                   |quantityOrdered|priceEach|orderDate     |storeAddress                             |
+-------+--------------------------+---------------+---------+--------------+-----------------------------------------+
|295665 |Macbook Pro Laptop        |1              |1700     |12/30/19 00:01|136 Church St, New York City, NY 10001   |
|295666 |LG Washing Machine        |1              |600.0    |12/29/19 07:03|562 2nd St, New York City, NY 10001      |
|295667 |USB-C Charging Cable      |1              |11.95    |12/12/19 18:21|277 Main St, New York City, NY 10001     |
|295668 |27in FHD Monitor          |1              |149.99   |12/22/19 15:13|410 6th St, San Francisco, CA 94016      |
|295669 |USB-C Charging Cable      |1              |11.95    |12/18/19 12:38|43 Hill St, Atlanta, GA 30301            |
|295670 |AA Batteries (4-pack)     |1   

In [0]:
sales_dataframe.printSchema()

root
 |-- orderId: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantityOrdered: string (nullable = true)
 |-- priceEach: string (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- storeAddress: string (nullable = true)



## Create the Database and Table

In [0]:
spark.sql('CREATE DATABASE IF NOT EXISTS SALES_DB')

Out[83]: DataFrame[]

In [0]:
spark.sql('USE SALES_DB')

Out[84]: DataFrame[]

In [0]:
sales_dataframe.columns

Out[85]: ['orderId',
 'product',
 'quantityOrdered',
 'priceEach',
 'orderDate',
 'storeAddress']

In [0]:
spark.sql('''CREATE TABLE IF NOT EXISTS SALES
          (ORDER_ID STRING,
          PRODUCT STRING,
          QUANTITY_ORDERED STRING,
          PRICE_EACH STRING,
          ORDER_DATE STRING,
          STORE_ADDRESS STRING
          )''')

Out[86]: DataFrame[]

In [0]:
spark.sql('describe SALES').show()

+----------------+---------+-------+
|        col_name|data_type|comment|
+----------------+---------+-------+
|        ORDER_ID|   string|   null|
|         PRODUCT|   string|   null|
|QUANTITY_ORDERED|   string|   null|
|      PRICE_EACH|   string|   null|
|      ORDER_DATE|   string|   null|
|   STORE_ADDRESS|   string|   null|
+----------------+---------+-------+



In [0]:
sales_dataframe.printSchema()

root
 |-- orderId: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantityOrdered: string (nullable = true)
 |-- priceEach: string (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- storeAddress: string (nullable = true)



## Create a temperory SQL Table from the DataFrame

In [0]:
sales_dataframe.createOrReplaceTempView('SALES_TEMP')

In [0]:
spark.sql('SELECT * FROM SALES_TEMP').show(10)

+-------+--------------------+---------------+---------+--------------+--------------------+
|orderId|             product|quantityOrdered|priceEach|     orderDate|        storeAddress|
+-------+--------------------+---------------+---------+--------------+--------------------+
| 295665|  Macbook Pro Laptop|              1|     1700|12/30/19 00:01|136 Church St, Ne...|
| 295666|  LG Washing Machine|              1|    600.0|12/29/19 07:03|562 2nd St, New Y...|
| 295667|USB-C Charging Cable|              1|    11.95|12/12/19 18:21|277 Main St, New ...|
| 295668|    27in FHD Monitor|              1|   149.99|12/22/19 15:13|410 6th St, San F...|
| 295669|USB-C Charging Cable|              1|    11.95|12/18/19 12:38|43 Hill St, Atlan...|
| 295670|AA Batteries (4-p...|              1|     3.84|12/31/19 22:58|200 Jefferson St,...|
| 295671|USB-C Charging Cable|              1|    11.95|12/16/19 15:10|928 12th St, Port...|
| 295672|USB-C Charging Cable|              2|    11.95|12/13/19 09:29

## Insert Data from the temperory Table to the SALES Table

In [0]:
spark.sql('''
          INSERT OVERWRITE SALES
          SELECT 
          orderId,
          product,
          quantityOrdered,
          priceEach,
          orderDate,
          storeAddress
          FROM SALES_TEMP''')

Out[91]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql('describe SALES').show()

+----------------+---------+-------+
|        col_name|data_type|comment|
+----------------+---------+-------+
|        ORDER_ID|   string|   null|
|         PRODUCT|   string|   null|
|QUANTITY_ORDERED|   string|   null|
|      PRICE_EACH|   string|   null|
|      ORDER_DATE|   string|   null|
|   STORE_ADDRESS|   string|   null|
+----------------+---------+-------+



In [0]:
spark.sql('SELECT * FROM SALES').show(10)

+--------+--------------------+----------------+----------+--------------+--------------------+
|ORDER_ID|             PRODUCT|QUANTITY_ORDERED|PRICE_EACH|    ORDER_DATE|       STORE_ADDRESS|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  295665|  Macbook Pro Laptop|               1|      1700|12/30/19 00:01|136 Church St, Ne...|
|  295666|  LG Washing Machine|               1|     600.0|12/29/19 07:03|562 2nd St, New Y...|
|  295667|USB-C Charging Cable|               1|     11.95|12/12/19 18:21|277 Main St, New ...|
|  295668|    27in FHD Monitor|               1|    149.99|12/22/19 15:13|410 6th St, San F...|
|  295669|USB-C Charging Cable|               1|     11.95|12/18/19 12:38|43 Hill St, Atlan...|
|  295670|AA Batteries (4-p...|               1|      3.84|12/31/19 22:58|200 Jefferson St,...|
|  295671|USB-C Charging Cable|               1|     11.95|12/16/19 15:10|928 12th St, Port...|
|  295672|USB-C Charging Cable|         

### Data Preparations using SQL

In [0]:
%sql

select * from sales
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001"
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035"
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001"
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001"
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001"


Lets remove the bad records

In [0]:
%sql 

select * from sales
where ORDER_ID is null
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null


In [0]:
%sql

select * from sales
where lower(ORDER_ID) = 'order id'
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [0]:
%sql

select * from sales
where ORDER_ID is not null 
and lower(ORDER_ID) <> 'order id'
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001"
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035"
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001"
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001"
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001"


## Extract the City and State from Store Address column

In [0]:
%sql

select *, split(STORE_ADDRESS,',')[1] as city
from sales
where ORDER_ID is not null 
and lower(ORDER_ID) <> 'order id'
limit 10


ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS,city
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001",New York City
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001",New York City
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",New York City
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",San Francisco
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",Atlanta
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001",New York City
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035",Portland
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001",Dallas
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001",Dallas
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001",Dallas


In [0]:
%sql

select *, 
split(STORE_ADDRESS,',')[1] as city,
substr(split(STORE_ADDRESS,',')[2],1,3) as state
from sales
where ORDER_ID is not null 
and lower(ORDER_ID) <> 'order id'
limit 10


ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS,city,state
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001",New York City,NY
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001",New York City,NY
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",New York City,NY
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",San Francisco,CA
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",Atlanta,GA
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001",New York City,NY
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035",Portland,OR
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001",Dallas,TX
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001",Dallas,TX
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001",Dallas,TX


## Aligning the format of order date and and extracting the year and month from them

In [0]:
%sql

select *, 
split(STORE_ADDRESS,',')[1] as city,
substr(split(STORE_ADDRESS,',')[2],1,3) as state,
to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm') as Date2,
year(to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm')) as sales_year,
month(to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm')) as sales_month
from sales
where ORDER_ID is not null 
and lower(ORDER_ID) <> 'order id'
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS,city,state,Date2,sales_year,sales_month
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001",New York City,NY,2019-12-30T00:01:00.000+0000,2019,12
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001",New York City,NY,2019-12-29T07:03:00.000+0000,2019,12
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",New York City,NY,2019-12-12T18:21:00.000+0000,2019,12
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",San Francisco,CA,2019-12-22T15:13:00.000+0000,2019,12
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",Atlanta,GA,2019-12-18T12:38:00.000+0000,2019,12
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001",New York City,NY,2019-12-31T22:58:00.000+0000,2019,12
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035",Portland,OR,2019-12-16T15:10:00.000+0000,2019,12
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001",Dallas,TX,2019-12-13T09:29:00.000+0000,2019,12
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001",Dallas,TX,2019-12-15T23:26:00.000+0000,2019,12
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001",Dallas,TX,2019-12-28T11:51:00.000+0000,2019,12


## Calculate revenue from each transactions

In [0]:
%sql

select *, 
split(STORE_ADDRESS,',')[1] as city,
substr(split(STORE_ADDRESS,',')[2],1,3) as state,
to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm') as Date2,
year(to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm')) as sales_year,
month(to_timestamp(ORDER_DATE,'MM/dd/yy HH:mm')) as sales_month,
(cast(QUANTITY_ORDERED as int) * cast(PRICE_EACH as float)) as revenue
from sales
where ORDER_ID is not null 
and lower(ORDER_ID) <> 'order id'
limit 10

ORDER_ID,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,ORDER_DATE,STORE_ADDRESS,city,state,Date2,sales_year,sales_month,revenue
295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001",New York City,NY,2019-12-30T00:01:00.000+0000,2019,12,1700.0
295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001",New York City,NY,2019-12-29T07:03:00.000+0000,2019,12,600.0
295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001",New York City,NY,2019-12-12T18:21:00.000+0000,2019,12,11.95
295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016",San Francisco,CA,2019-12-22T15:13:00.000+0000,2019,12,149.99
295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301",Atlanta,GA,2019-12-18T12:38:00.000+0000,2019,12,11.95
295670,AA Batteries (4-pack),1,3.84,12/31/19 22:58,"200 Jefferson St, New York City, NY 10001",New York City,NY,2019-12-31T22:58:00.000+0000,2019,12,3.84
295671,USB-C Charging Cable,1,11.95,12/16/19 15:10,"928 12th St, Portland, OR 97035",Portland,OR,2019-12-16T15:10:00.000+0000,2019,12,11.95
295672,USB-C Charging Cable,2,11.95,12/13/19 09:29,"813 Hickory St, Dallas, TX 75001",Dallas,TX,2019-12-13T09:29:00.000+0000,2019,12,23.9
295673,Bose SoundSport Headphones,1,99.99,12/15/19 23:26,"718 Wilson St, Dallas, TX 75001",Dallas,TX,2019-12-15T23:26:00.000+0000,2019,12,99.99
295674,AAA Batteries (4-pack),4,2.99,12/28/19 11:51,"77 7th St, Dallas, TX 75001",Dallas,TX,2019-12-28T11:51:00.000+0000,2019,12,11.96


##Aligning the necessary datatype convertions

In [0]:
%sql

SELECT 
    CAST(ORDER_ID AS INT) AS ORDER_ID,
    TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm') AS ORDER_DATE,
    YEAR(TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm')) AS SALES_YEAR,
    MONTH(TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm')) AS SALES_MONTH,
    STORE_ADDRESS,
    SPLIT(STORE_ADDRESS, ',')[1] AS CITY,
    SUBSTR(SPLIT(STORE_ADDRESS, ',')[2], 1, 3) AS STATE,
    PRODUCT,
    CAST(QUANTITY_ORDERED AS INTEGER) AS QUANTITY_ORDERED,
    CAST(PRICE_EACH AS FLOAT) AS PRICE_EACH,
    CAST(QUANTITY_ORDERED AS INT) * CAST(PRICE_EACH AS FLOAT) AS REVENUE
FROM 
    sales
WHERE 
    ORDER_ID IS NOT NULL 
    AND LOWER(ORDER_ID) <> 'order id'
LIMIT 10


ORDER_ID,ORDER_DATE,SALES_YEAR,SALES_MONTH,STORE_ADDRESS,CITY,STATE,PRODUCT,QUANTITY_ORDERED,PRICE_EACH,REVENUE
295665,2019-12-30T00:01:00.000+0000,2019,12,"136 Church St, New York City, NY 10001",New York City,NY,Macbook Pro Laptop,1,1700.0,1700.0
295666,2019-12-29T07:03:00.000+0000,2019,12,"562 2nd St, New York City, NY 10001",New York City,NY,LG Washing Machine,1,600.0,600.0
295667,2019-12-12T18:21:00.000+0000,2019,12,"277 Main St, New York City, NY 10001",New York City,NY,USB-C Charging Cable,1,11.95,11.95
295668,2019-12-22T15:13:00.000+0000,2019,12,"410 6th St, San Francisco, CA 94016",San Francisco,CA,27in FHD Monitor,1,149.99,149.99
295669,2019-12-18T12:38:00.000+0000,2019,12,"43 Hill St, Atlanta, GA 30301",Atlanta,GA,USB-C Charging Cable,1,11.95,11.95
295670,2019-12-31T22:58:00.000+0000,2019,12,"200 Jefferson St, New York City, NY 10001",New York City,NY,AA Batteries (4-pack),1,3.84,3.84
295671,2019-12-16T15:10:00.000+0000,2019,12,"928 12th St, Portland, OR 97035",Portland,OR,USB-C Charging Cable,1,11.95,11.95
295672,2019-12-13T09:29:00.000+0000,2019,12,"813 Hickory St, Dallas, TX 75001",Dallas,TX,USB-C Charging Cable,2,11.95,23.9
295673,2019-12-15T23:26:00.000+0000,2019,12,"718 Wilson St, Dallas, TX 75001",Dallas,TX,Bose SoundSport Headphones,1,99.99,99.99
295674,2019-12-28T11:51:00.000+0000,2019,12,"77 7th St, Dallas, TX 75001",Dallas,TX,AAA Batteries (4-pack),4,2.99,11.96


## Converting the Pure SQL into a dataframe using the Spark SQL

In [0]:
sales_clean_dataframe = spark.sql('''
                                    SELECT 
                                        CAST(ORDER_ID AS INT) AS ORDER_ID,
                                        TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm') AS ORDER_DATE,
                                        YEAR(TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm')) AS SALES_YEAR,
                                        MONTH(TO_TIMESTAMP(ORDER_DATE, 'MM/dd/yy HH:mm')) AS SALES_MONTH,
                                        STORE_ADDRESS,
                                        SPLIT(STORE_ADDRESS, ',')[1] AS CITY,
                                        SUBSTR(SPLIT(STORE_ADDRESS, ',')[2], 1, 3) AS STATE,
                                        PRODUCT,
                                        CAST(QUANTITY_ORDERED AS INTEGER) AS QUANTITY_ORDERED,
                                        CAST(PRICE_EACH AS FLOAT) AS PRICE_EACH,
                                        CAST(QUANTITY_ORDERED AS INT) * CAST(PRICE_EACH AS FLOAT) AS REVENUE
                                    FROM 
                                        sales
                                    WHERE 
                                        ORDER_ID IS NOT NULL 
                                        AND LOWER(ORDER_ID) <> 'order id'
                                                ''')

In [0]:
sales_clean_dataframe.show(10)

+--------+-------------------+----------+-----------+--------------------+--------------+-----+--------------------+----------------+----------+-------+
|ORDER_ID|         ORDER_DATE|SALES_YEAR|SALES_MONTH|       STORE_ADDRESS|          CITY|STATE|             PRODUCT|QUANTITY_ORDERED|PRICE_EACH|REVENUE|
+--------+-------------------+----------+-----------+--------------------+--------------+-----+--------------------+----------------+----------+-------+
|  295665|2019-12-30 00:01:00|      2019|         12|136 Church St, Ne...| New York City|   NY|  Macbook Pro Laptop|               1|    1700.0| 1700.0|
|  295666|2019-12-29 07:03:00|      2019|         12|562 2nd St, New Y...| New York City|   NY|  LG Washing Machine|               1|     600.0|  600.0|
|  295667|2019-12-12 18:21:00|      2019|         12|277 Main St, New ...| New York City|   NY|USB-C Charging Cable|               1|     11.95|  11.95|
|  295668|2019-12-22 15:13:00|      2019|         12|410 6th St, San F...| San Fra

In [0]:
sales_clean_dataframe.printSchema()

root
 |-- ORDER_ID: integer (nullable = true)
 |-- ORDER_DATE: timestamp (nullable = true)
 |-- SALES_YEAR: integer (nullable = true)
 |-- SALES_MONTH: integer (nullable = true)
 |-- STORE_ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- PRODUCT: string (nullable = true)
 |-- QUANTITY_ORDERED: integer (nullable = true)
 |-- PRICE_EACH: float (nullable = true)
 |-- REVENUE: float (nullable = true)



## Create a temperary sales clean sql table from the sales_clean_dataframe

In [0]:
sales_clean_dataframe.createOrReplaceTempView('sales_clean_table_temp')

In [0]:
spark.sql('select * from sales_clean_table_temp limit 10').show(10)

+--------+-------------------+----------+-----------+--------------------+--------------+-----+--------------------+----------------+----------+-------+
|ORDER_ID|         ORDER_DATE|SALES_YEAR|SALES_MONTH|       STORE_ADDRESS|          CITY|STATE|             PRODUCT|QUANTITY_ORDERED|PRICE_EACH|REVENUE|
+--------+-------------------+----------+-----------+--------------------+--------------+-----+--------------------+----------------+----------+-------+
|  295665|2019-12-30 00:01:00|      2019|         12|136 Church St, Ne...| New York City|   NY|  Macbook Pro Laptop|               1|    1700.0| 1700.0|
|  295666|2019-12-29 07:03:00|      2019|         12|562 2nd St, New Y...| New York City|   NY|  LG Washing Machine|               1|     600.0|  600.0|
|  295667|2019-12-12 18:21:00|      2019|         12|277 Main St, New ...| New York City|   NY|USB-C Charging Cable|               1|     11.95|  11.95|
|  295668|2019-12-22 15:13:00|      2019|         12|410 6th St, San F...| San Fra

## Create a Table (`sales_clean_table`) in the sales_db database to hold the data permennatly in the system

In [0]:
spark.sql("DROP TABLE IF EXISTS sales_clean_table")

Out[111]: DataFrame[]

In [0]:
spark.sql('''
    CREATE TABLE sales_clean_table (
        ORDER_ID INTEGER,
        ORDER_DATE TIMESTAMP,
        SALES_YEAR INTEGER,
        SALES_MONTH INTEGER,
        STORE_ADDRESS STRING,
        CITY STRING,
        STATE STRING,
        PRODUCT STRING,
        QUANTITY_ORDERED INTEGER,
        PRICE_EACH FLOAT,
        REVENUE FLOAT
    )
    USING PARQUET
    PARTITIONED BY (SALES_YEAR, SALES_MONTH)
    OPTIONS ('compression'='snappy')
    LOCATION 'dbfs:/FileStore/tables/salesdata/published'
''')

Out[112]: DataFrame[]

In [0]:
spark.sql('describe sales_clean_table').show(truncate= False)

+-----------------------+---------+-------+
|col_name               |data_type|comment|
+-----------------------+---------+-------+
|ORDER_ID               |int      |null   |
|ORDER_DATE             |timestamp|null   |
|STORE_ADDRESS          |string   |null   |
|CITY                   |string   |null   |
|STATE                  |string   |null   |
|PRODUCT                |string   |null   |
|QUANTITY_ORDERED       |int      |null   |
|PRICE_EACH             |float    |null   |
|REVENUE                |float    |null   |
|SALES_YEAR             |int      |null   |
|SALES_MONTH            |int      |null   |
|# Partition Information|         |       |
|# col_name             |data_type|comment|
|SALES_YEAR             |int      |null   |
|SALES_MONTH            |int      |null   |
+-----------------------+---------+-------+



**Important** : Note that the order of the column in the sales_clean_table has a difference from its creation because the columns used for partition information come to the end, keep this order change in mind (look at the describe table name - query to see the actual order) while inserting the values into the table.

In [0]:
spark.sql('describe sales_clean_table_temp').show(truncate = False)

+----------------+---------+-------+
|col_name        |data_type|comment|
+----------------+---------+-------+
|ORDER_ID        |int      |null   |
|ORDER_DATE      |timestamp|null   |
|SALES_YEAR      |int      |null   |
|SALES_MONTH     |int      |null   |
|STORE_ADDRESS   |string   |null   |
|CITY            |string   |null   |
|STATE           |string   |null   |
|PRODUCT         |string   |null   |
|QUANTITY_ORDERED|int      |null   |
|PRICE_EACH      |float    |null   |
|REVENUE         |float    |null   |
+----------------+---------+-------+



## Inserting the Temporary Table data into the permenant Table -- > sales_clean_table

In [0]:
spark.sql('''
    INSERT OVERWRITE TABLE sales_clean_table
    SELECT
        ORDER_ID,
        ORDER_DATE,
        STORE_ADDRESS,
        CITY,
        STATE,
        PRODUCT,
        QUANTITY_ORDERED,
        PRICE_EACH,
        REVENUE,
        SALES_YEAR,
        SALES_MONTH
    FROM
        sales_clean_table_temp
''')

Out[115]: DataFrame[]

In [0]:
spark.sql('select * from sales_clean_table limit 10').show(10)

+--------+-------------------+--------------------+--------------+-----+--------------------+----------------+----------+-------+----------+-----------+
|ORDER_ID|         ORDER_DATE|       STORE_ADDRESS|          CITY|STATE|             PRODUCT|QUANTITY_ORDERED|PRICE_EACH|REVENUE|SALES_YEAR|SALES_MONTH|
+--------+-------------------+--------------------+--------------+-----+--------------------+----------------+----------+-------+----------+-----------+
|  295665|2019-12-30 00:01:00|136 Church St, Ne...| New York City|   NY|  Macbook Pro Laptop|               1|    1700.0| 1700.0|      2019|         12|
|  295666|2019-12-29 07:03:00|562 2nd St, New Y...| New York City|   NY|  LG Washing Machine|               1|     600.0|  600.0|      2019|         12|
|  295667|2019-12-12 18:21:00|277 Main St, New ...| New York City|   NY|USB-C Charging Cable|               1|     11.95|  11.95|      2019|         12|
|  295668|2019-12-22 15:13:00|410 6th St, San F...| San Francisco|   CA|    27in F

In [0]:
spark.sql('select count(*) from sales_clean_table').show()

+--------+
|count(1)|
+--------+
|  496583|
+--------+



In [0]:
spark.sql('describe sales_clean_table').show()  

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            ORDER_ID|      int|   null|
|          ORDER_DATE|timestamp|   null|
|       STORE_ADDRESS|   string|   null|
|                CITY|   string|   null|
|               STATE|   string|   null|
|             PRODUCT|   string|   null|
|    QUANTITY_ORDERED|      int|   null|
|          PRICE_EACH|    float|   null|
|             REVENUE|    float|   null|
|          SALES_YEAR|      int|   null|
|         SALES_MONTH|      int|   null|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|          SALES_YEAR|      int|   null|
|         SALES_MONTH|      int|   null|
+--------------------+---------+-------+

